In [77]:
import pickle
import networkx as nx
import numpy as np
import pandas as pd
# import math
# import EoN
# import random
import pickle
from Phage_api import settings_local as local_settings
import glob
# import gseapy as gp

In [78]:
import importlib.util
gseapy=importlib.util.spec_from_file_location("gseapy","/home/platform/project/crustdb_platform/crustdb_api/workspace/analysis_script/scripts/cytotopo_reference_package/gseapy/plot.py")
DotPlot = importlib.util.module_from_spec(gseapy)
gseapy.loader.exec_module(DotPlot)

In [79]:
home = local_settings.CRUSTDB_DATABASE+'topology/'
home

'/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/'

In [80]:
def run_go_analysis(go_df):
    result_new = go_df
    if result_new.shape[0] > 200:
        figsize=False
        size = False
        top_term=3
    else:
        figsize = (5,10)
        size = 50
        top_term = 5

    x_key = 'Gene_set'
    if not figsize:
        figsize = (int(len(result_new[x_key].unique())/3), int(top_term*len(result_new[x_key].unique())/10))
    if not size:
        size = int(top_term*len(result_new[x_key].unique())/15)

    result_new.loc[:,'Term'] = result_new.loc[:,'Term'].str.split('(', expand=True)[0]

    cutoff = 0.01
    while int(len(result_new[result_new.loc[:,'Adjusted P-value'] <= cutoff][x_key].unique())) < 3:
        cutoff = cutoff + 0.01

    dot = DotPlot.DotPlot(
        df=result_new,
        x='Gene_set',
        y='Term',
        x_order=False,
        y_order=False,
        hue="Adjusted P-value",
        title="title",
        thresh=cutoff,
        n_terms=int(top_term),
        dot_scale=size,
        figsize=figsize,
        cmap="viridis_r",
        ofname=None,
        marker='o',
    )
    go_info = dot.data[['Gene_set', 'Term', 'p_inv', 'Hits_ratio']]
    return go_info

In [81]:
def generate_go_info(species, conformation, algo, param):
    go_df = pd.read_csv(home+species+'/'+conformation+'/'+algo+'/'+param+'/Go.csv', index_col=0)
    go_info = run_go_analysis(go_df)
    go_info.to_csv(home+species+'/'+conformation+'/'+algo+'/'+param+'/Go_result.csv')

In [82]:
layer1 = glob.glob(home+'*')
for _species in layer1:
    species = _species.split('/')[-1]
    if species not in ['Mice']:
        continue
    layer2 = glob.glob(_species+'/*')
    for _conformation in layer2:
        conformation = _conformation.split('/')[-1]
        try:
            for algo in ['1NN', 'KNN', 'KNN_SNN', 'RNN', 'RNN_SNN']:
                if algo in ['1NN']:
                    param = '/'
                    generate_go_info(species, conformation, algo, param)
                else:
                    _algo = _conformation + '/' + algo
                    layer3 = glob.glob(_algo + '/*')
                    for _param in layer3:
                        if 'pkl' in _param:
                            continue
                        param = _param.split('/')[-1]
                        generate_go_info(species, conformation, algo, param)
        except:
            print('Error ---> '+species+'/'+conformation+'/'+algo)